In [1]:
# Packages
from IPython.display import Image
import rasterio
import skimage
import os
import sys
import pathlib
import math
import functools
from skimage import exposure
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from rasterio.plot import show
from osgeo import gdal

In [120]:
# Get absolute file paths. Returns generator object
def absoluteFilePaths(directory):
    for dirpath,_,filenames in os.walk(directory):
       for f in filenames:
           yield os.path.abspath(os.path.join(dirpath, f))

# Normalize array
def normalize(arr):
    ''' Function to normalize an input array to 0-1 '''
    arr_max = arr.max()
    return arr / arr_max

# Reorder Planet scenes to RGB
def reorder_to_rgb(image):
    '''reorders planet bands to red, green, blue for imshow'''
    blue = normalize(image[:,:,0])
    green = normalize(image[:,:,1])
    red = normalize(image[:,:,2])
    return np.stack([red, green, blue], axis=-1)

# Contrast stretching algorithm for multiband images
def contrast_stretch_mb(img):
    # Loop over RGB bands
    for b in range(0,3):
        p2, p98 = np.percentile(img[:,:,b], (2, 98))
        img_scaled = exposure.rescale_intensity(img, in_range=(p2, p98))
        img[:,:,b] = img_scaled[:,:,b]
    return img

In [2]:

def planet2chips(tiff_directory, chip_directory):
    
    """ Creates image chips (GeoTiffs and PNGs) of a GeoTiff file in a 
    specified directory and saves in new directory location 
    """
  
    # Get all analytic SR GeoTiff filnames in specified directory
    files = np.array(os.listdir(tiff_directory))
    tiff = pd.Series(files).str.contains('SR.tif')
    file = files[tiff][0]

    # Get image name to use for creating directory
    image_name = file.split("_")[0:3]
    image_name = "%s_%s_%s" % (image_name[0], image_name[1], image_name[2])

    # Image chip destination directory and subdirectories
    image_dir = os.path.join(chip_directory, image_name)   

    chip_dir = os.path.join(image_dir,'chips')
    png_dir = os.path.join(image_dir, 'pngs')

    # Print filenames
    print('filename: ' + file + '\n' + 'image name: ' + image_name)

    # Make directories to store raw and rgb image chips
    pathlib.Path(chip_dir).mkdir(parents=True, exist_ok=True)
    pathlib.Path(png_dir).mkdir(parents=True, exist_ok=True)
    
    # TODO Read in full image and calculate percentiles for contrast stretching

    # Iterate over image blocks - which are 256x256 - and save new GeoTiffs
    with rasterio.open(os.path.join(tiff_directory, file)) as src:
        # Get block dimensions of src
        for ji, window in src.block_windows(1):
            r = src.read((1,2,3,4), window=window)
            if 0 in r:
                print("Skipped chip due to missing data.\n")

            else:
                # Scale variable. Note bands of Planet imagery go BGR
                b = src.read((3,2,1), window=window)
                b = exposure.rescale_intensity(b, out_range='uint8')

                # Open a new GeoTiff data file in which to save the raw image chip
                with rasterio.open((chip_dir + '/' + image_name + '_' + str(ji[0]) + '_' + str(ji[1]) + '.tif'), 'w', driver='GTiff',
                           height=r.shape[1], width=r.shape[2], count=4,
                           dtype=rasterio.uint16, crs=src.crs, 
                           transform=src.transform) as new_img:

                    # Write the raw image to the new GeoTiff
                    new_img.write(r)

                # Open a new PNG data file in which to save the rescaled RGB image chip
                with rasterio.open((png_dir + '/' + image_name + '_' + str(ji[0]) + '_' + str(ji[1]) + '.png'), 'w', driver='PNG',
                           height=r.shape[1], width=r.shape[2], count=3,
                           dtype=rasterio.ubyte, crs=src.crs, 
                           transform=src.transform) as new_img:

                    # Write the rescaled image chip to the new PNG
                    new_img.write(b.astype('uint8'))

In [3]:
def process_planet_orders(source_dir, target_dir):
    
    """Find unique PlanetScope scenes in a directory of Planet order folders
    and process newly added scenes into image chips"""
    
    # Get list of all planet orders in source directory
    orders = np.array(next(os.walk(source_dir))[1])
    # Add full path to each order directory
    orders = [os.path.join(source_dir, o) for o in orders]
    
    scenes = []
    scene_paths = []
    
    for o in orders:
        # scenes in order
        s_ids = np.array(next(os.walk(o))[1])
        s_ids_paths = [os.path.join(source_dir,o,s) for s in s_ids]
        
        # add to lists
        scenes.append(s_ids)
        scene_paths.append(s_ids_paths)
    
    # Flatten lists
    scenes = list(np.concatenate(scenes))
    scene_paths = list(np.concatenate(scene_paths))
    
    # Check which scenes already have chip folders
    scenes_exist = np.array(next(os.walk(target_dir))[1])
    
    # Remove scenes that already exist from list of scenes to process
    for s, sp in zip(scenes, scene_paths):
        if s in scenes_exist:
            scenes.remove(s)
            scene_paths.remove(sp) 
    
    # Apply GeoTiff chipping function to each unprocessed scene
    for sp in scene_paths:
        planet2chips(tiff_directory = sp, chip_directory = target_dir)    

Apply to a test image to check performance

Apply the function to process all Planet orders presently in Box

In [123]:
# Run function
sdir = '/Users/Tyler-SFG/Desktop/Box Sync/SFG Centralized Resources/Projects/Aquaculture/Waitt Aquaculture/aqua-mapping/aqua-mapping-data/aqua-images/planet'
tdir = '/Users/Tyler-SFG/Desktop/Box Sync/SFG Centralized Resources/Projects/Aquaculture/Waitt Aquaculture/aqua-mapping/aqua-mapping-data/aqua-images/planet_chips'

# os.path.isdir(sdir)
process_planet_orders(sdir, tdir)    

42
filename: 20180410_020422_0f31_3B_AnalyticMS_SR.tif
image name: 20180410_020422_0f31
Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Sk

/Users/Tyler-SFG/anaconda/envs/planet/lib/python3.6/site-packages/ipykernel/__main__.py:41: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
/Users/Tyler-SFG/anaconda/envs/planet/lib/python3.6/site-packages/rasterio/__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to missing data.

Skipped chip due to 